In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scripts.utils import save_fig

%load_ext autoreload
%autoreload 2
plt.rcParams["figure.figsize"] = (12, 4)
font = {'weight' : 'regular',
        'family': "serif",
        'size'   : 16}
plt.rc('font', **font)
plt.rc('xtick', labelsize='14')
plt.rc('ytick', labelsize='14')

In [ ]:
COLOR_PALETTE = np.array(sns.color_palette("colorblind"))[[5,0,2,3]]

In [ ]:
prices = pd.read_excel('data/prices/prices_06-2023.xlsx', header=0, index_col=0)
# prices = prices.drop(list(range(2014, 2021)), axis=1)

In [ ]:
prices = prices.reset_index().groupby(['index', 'Appellation']).first()
prices.index.names = ['Vineyard', 'Appellation']

In [ ]:
(
    prices.reset_index()[["Vineyard", "Appellation"]].sort_values(
        ["Appellation", "Vineyard"]
    )
).to_excel("data/prices/vineyard_list.xlsx", index=False)

In [ ]:
prices = (
    prices.transpose()
    .reset_index()
    .melt(id_vars="index")
    .rename(columns={"index": "Vintage", "value": "Price"})
)

In [ ]:
prices = prices[prices['Vintage'] >= 1960]

In [ ]:
ranking = prices.groupby('Vineyard')['Price'].mean().sort_values(ascending=False)
top_wines = ranking.index.to_list()

In [ ]:
plt.figure(figsize=(10, 6))
plt.yscale("log")
PROPS = {
    "boxprops": {"facecolor": "lightgrey", "edgecolor": "k"},
    "medianprops": {"color": "k"},
    "whiskerprops": {"color": "k"},
    "capprops": {"color": "k"},
}
plt.grid(True, which="major")
sns.boxplot(
    data=prices[(prices["Vineyard"].isin(top_wines[:20]))],
    x="Vineyard",
    y="Price",
    order=top_wines[:20],
    **PROPS
)
plt.xticks(rotation=90)
plt.ylim(0, 5000)
plt.ylabel("Average 2023 price for vintages 1960-2013(€)")

save_fig("views/prices/top_wines", width_column="1.5")
plt.show()

# Get appellation price indexes

## Médoc

In [ ]:
prices['Appellation'].unique()

In [ ]:
prices_medoc = prices[prices["Appellation"] == "Médoc"]
print("Missing per vineyard:")
display(prices_medoc[prices_medoc.isna().any(axis=1)]["Vineyard"].value_counts())

EXCLUDED_VINEYARDS = [
    "Château Clerc-Milon",
    "Château Latour",
    "Château Pontet-Canet",
    "Château Pichon-Longueville Comtesse de Lalande",
    "Château Léoville-Barton",
    "Château d'Armailhac",
]
wines_medoc = [
    wine
    for wine in list(prices_medoc["Vineyard"].unique())
    if wine not in EXCLUDED_VINEYARDS
]
prices_medoc = prices_medoc[~prices_medoc["Vineyard"].isin(EXCLUDED_VINEYARDS)]
display(
    prices_medoc[prices_medoc.isna().any(axis=1)]["Vintage"].value_counts().sort_index()
)

In [ ]:
plt.figure(figsize=(14, 8))
plt.yscale("log")
sns.lineplot(data=prices_medoc, x="Vintage", y="Price", hue="Vineyard")
plt.suptitle("Prices per vintage")
plt.legend(bbox_to_anchor=(1.01, 1), borderaxespad=0)
plt.show()

### Convert prices to log, then interpolate linearly

In [ ]:
prices_medoc.loc[:,'Price'] = np.log(prices_medoc['Price'])
prices_medoc.loc[:,'Price'] = prices_medoc.groupby('Vineyard').transform(lambda s: s.interpolate(limit_area='inside'))#.ffill()

In [ ]:
plt.figure(figsize=(14, 8))
sns.lineplot(
    data=prices_medoc, x="Vintage", y="Price", hue="Vineyard", palette="colorblind"
)
plt.suptitle("Prices per vintage")
plt.legend(bbox_to_anchor=(1.01, 1), borderaxespad=0)
plt.tight_layout()
plt.show()

In [ ]:
avg_price_medoc = prices_medoc.groupby('Vintage').mean()

## Pessac-Léognan

In [ ]:
prices_pessac = prices[prices['Appellation'] == 'Pessac-Léognan']
print('Missing per vineyard:')
display(prices_pessac[prices_pessac.isna().any(axis=1)]['Vineyard'].value_counts())

EXCLUDED_VINEYARDS = [
    'Château La Tour Haut-Brion',
    'Château Les Carmes Haut-Brion',
    'Château Haut-Bailly',
]
wines_pessac = [wine for wine in list(prices_pessac['Vineyard'].unique()) if wine not in EXCLUDED_VINEYARDS]

prices_pessac = prices_pessac[~prices_pessac['Vineyard'].isin(EXCLUDED_VINEYARDS)]
display(prices_pessac[prices_pessac.isna().any(axis=1)]['Vintage'].value_counts().sort_index())

In [ ]:
plt.figure(figsize=(14,8))
plt.yscale('log')
sns.lineplot(data=prices_pessac, x='Vintage', y='Price', hue='Vineyard')
plt.suptitle('Prices per vintage')
plt.legend(bbox_to_anchor=(1.01, 1),
           borderaxespad=0)
plt.show()

### Convert prices to log, then interpolate linearly

In [ ]:
prices_pessac.loc[:,'Price'] = np.log(prices_pessac['Price'])
prices_pessac.loc[:,'Price'] = prices_pessac.groupby('Vineyard').transform(lambda s: s.interpolate(limit_area='inside'))#.ffill()

In [ ]:
plt.figure(figsize=(14,8))
sns.lineplot(data=prices_pessac, x='Vintage', y='Price', hue='Vineyard')
plt.suptitle('Prices per vintage')
plt.legend(bbox_to_anchor=(1.01, 1),
           borderaxespad=0)
plt.show()

In [ ]:
avg_price_pessac = prices_pessac.groupby('Vintage').mean()

## Saint-Emilion

In [ ]:
prices_st_em = prices[prices["Appellation"] == "Saint-Emilion"]
print("Missing per vineyard:")
display(prices_st_em[prices_st_em.isna().any(axis=1)]["Vineyard"].value_counts())

EXCLUDED_VINEYARDS = [
    "Château Pavie-Macquin",
    "Château Beauséjour",
    "Château Beau-Séjour Bécot",
    "Château Magdelaine",
    "Château Belair-Monange",
    "Château Larcis-Ducasse",
]
wines_st_em = [
    wine
    for wine in list(prices_st_em["Vineyard"].unique())
    if wine not in EXCLUDED_VINEYARDS
]

prices_st_em = prices_st_em[~prices_st_em["Vineyard"].isin(EXCLUDED_VINEYARDS)]
display(
    prices_st_em[prices_st_em.isna().any(axis=1)]["Vintage"].value_counts().sort_index()
)

In [ ]:
plt.figure(figsize=(14,8))
plt.yscale('log')
sns.lineplot(data=prices_st_em, x='Vintage', y='Price', hue='Vineyard')
plt.suptitle('Prices per vintage')
plt.legend(bbox_to_anchor=(1.01, 1),
           borderaxespad=0)
plt.show()

### Convert prices to log, then interpolate linearly

In [ ]:
prices_st_em.loc[:,'Price'] = np.log(prices_st_em['Price'])
prices_st_em.loc[:,'Price'] = prices_st_em.groupby('Vineyard').transform(lambda s: s.interpolate(limit_area='inside'))#.ffill()

In [ ]:
prices_st_em.loc[prices_st_em['Vineyard'] == 'Château Angélus']

In [ ]:
plt.figure(figsize=(14, 8))
sns.lineplot(data=prices_st_em, x="Vintage", y="Price", hue="Vineyard")
plt.suptitle("Prices per vintage")
plt.legend(bbox_to_anchor=(1.01, 1), borderaxespad=0)
plt.show()

In [ ]:
avg_price_st_em = prices_st_em.groupby('Vintage').mean()

## Pomerol

In [ ]:
prices_pomerol = prices[prices["Appellation"] == "Pomerol"]
print("Missing per vineyard:")
display(prices_pomerol[prices_pomerol.isna().any(axis=1)]["Vineyard"].value_counts())

EXCLUDED_VINEYARDS = [
    "Château Le Gay",
    "Château Le Bon pasteur",
    "Clos l'Eglise",
    "Château l'Eglise Clinet",
    "Château La Fleur-Pétrus",
]
wines_pomerol = [
    wine
    for wine in list(prices_pomerol["Vineyard"].unique())
    if wine not in EXCLUDED_VINEYARDS
]

prices_pomerol = prices_pomerol[~prices_pomerol["Vineyard"].isin(EXCLUDED_VINEYARDS)]
display(
    prices_pomerol[prices_pomerol.isna().any(axis=1)]["Vintage"]
    .value_counts()
    .sort_index()
)

In [ ]:
plt.figure(figsize=(14,8))
plt.yscale('log')
sns.lineplot(data=prices_pomerol, x='Vintage', y='Price', hue='Vineyard')
plt.suptitle('Prices per vintage')
plt.legend(bbox_to_anchor=(1.01, 1),
           borderaxespad=0)
plt.show()

### Convert prices to log, then interpolate linearly

In [ ]:
prices_pomerol.loc[:,'Price'] = np.log(prices_pomerol['Price'])
prices_pomerol.loc[:,'Price'] = prices_pomerol.groupby('Vineyard').transform(lambda s: s.interpolate(limit_area='inside'))#.ffill()

In [ ]:
plt.figure(figsize=(14, 8))
sns.lineplot(data=prices_pomerol, x="Vintage", y="Price", hue="Vineyard")
plt.suptitle("Prices per vintage")
plt.legend(bbox_to_anchor=(1.01, 1), borderaxespad=0)
plt.show()

In [ ]:
avg_price_pomerol = prices_pomerol.groupby('Vintage').mean()

# Export individual wine prices

In [ ]:
prices_per_vineyard = pd.concat([prices_medoc, prices_pessac, prices_pomerol, prices_st_em], axis=0)

In [ ]:
prices_per_vineyard.to_excel('data/prices/prices_per_vineyard.xlsx')
prices_per_vineyard.loc[prices_per_vineyard['Vineyard'] == 'Château Angélus']

In [ ]:
plt.rc("text", usetex=False)
plt.figure(figsize=(14, 7))

plt.yscale("log")
plt.grid(True, which="both", axis="both", alpha=0.5)

sns.lineplot(
    data=prices_per_vineyard,
    x="Vintage",
    y="Price",
    style="Appellation",
    hue="Appellation",
    linewidth=2,
    dashes=[(1, 1), (3, 2), (1, 1, 4, 1), ()],
    markers=["s", "o", "o", "s"],
    palette=COLOR_PALETTE,
    markeredgecolor="k",
)
plt.ylabel("2021 price(€)")
plt.xticks(range(1960, 2018, 5))
save_fig("views/prices/evolution_by_vineyard", "1.5")
plt.show()

## Aggregate all prices

In [ ]:
prices_agg = pd.concat([avg_price_medoc, avg_price_pessac, avg_price_pomerol, avg_price_st_em], axis=1)
prices_agg.columns = ['Médoc', 'Pessac-Léognan', 'Pomerol', 'Saint-Emilion']

In [ ]:
prices_agg.to_excel('data/prices/generated_agg_prices_per_region.xlsx')

## Visualize prices per region

In [ ]:
prices_agg.head()

In [ ]:
prices_melt = (
    prices_agg.reset_index()
    .melt(id_vars="Vintage")
    .rename(columns={"variable": "Appellation", "value": "Price"})
)

In [ ]:
prices_melt['Price'] = np.exp(prices_melt['Price'])

In [ ]:
import matplotlib.ticker as mticker

plt.figure(figsize=(14, 7))

plt.grid(True, which="both", axis="both", alpha=0.5)

sns.lineplot(
    data=prices_melt.sort_values("Appellation"),
    x="Vintage",
    y="Price",
    style="Appellation",
    hue="Appellation",
    linewidth=2,
    dashes=[(1, 1), (3, 2), (1, 1, 4, 1), ()],
    markers=["s", "o", "o", "s"],
    palette=COLOR_PALETTE,
    markeredgecolor="k",
    markersize=8,
)
plt.ylabel("2021 price(€)", labelpad=10)
plt.xticks(range(1960, 2014, 5))
plt.yscale("log")
plt.gca().yaxis.set_major_formatter(mticker.ScalarFormatter())
plt.gca().yaxis.set_minor_formatter(mticker.NullFormatter())
plt.gca().set_yticks([50, 100, 200, 400, 800])

save_fig("views/prices/evolution_by_appellation", width_column="1.5")

plt.show()

### Show skew

In [ ]:
import numpy as np
from scipy.stats import kurtosis, skew

skew_table = pd.DataFrame(index=["Skew", "Kurtosis"], columns=prices_agg.columns)
skew_table.loc["Skew", :] = prices_agg.apply(lambda x: skew(x), axis=0)
skew_table.loc["Kurtosis", :] = prices_agg.apply(lambda x: kurtosis(x), axis=0)
skew_table

In [ ]:
prices_agg_exp = np.exp(prices_agg)
skew_table = pd.DataFrame(index=["Skew", "Kurtosis"], columns=prices_agg.columns)
skew_table.loc["Skew", :] = prices_agg_exp.apply(lambda x: skew(x), axis=0)
skew_table.loc["Kurtosis", :] = prices_agg_exp.apply(lambda x: kurtosis(x), axis=0)
skew_table

In [ ]:
prices_agg_log = np.log(prices_agg - 3.7)
skew_table = pd.DataFrame(index=["Skew", "Kurtosis"], columns=prices_agg.columns)
skew_table.loc["Skew", :] = prices_agg_log.apply(lambda x: skew(x), axis=0)
skew_table.loc["Kurtosis", :] = prices_agg_log.apply(lambda x: kurtosis(x), axis=0)
skew_table

---
# End of notebook
